In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224, usb=0)

image = widgets.Image(format='jpeg', width=400, height=400)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
classfication_result_image = widgets.Textarea(
            value=u'Result:=',
            description=u'Classifcation',
            disabled=False)



In [2]:
import cv2
import numpy as np


def preprocess(camera_value):
    x = camera_value
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = np.asarray(x)
    x = (x.astype(np.float32) / 127.0) - 1
    return x

In [3]:
import tflite_runtime.interpreter as tflite     #引入tflite模組

interpreter = tflite.Interpreter(model_path='model_unquant.tflite')

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']

In [4]:
import numpy as np
data, target = np.array_split(np.loadtxt('labels.txt', dtype=str), [-1], axis=1)

label=target.flatten().tolist()

In [5]:
display(widgets.VBox([
    widgets.HBox([image, classfication_result_image]),
]))

In [6]:
def execute(change):
    image = change['new']
    classfication_result_image.value = ' '
    interpreter.allocate_tensors()
    input_tensor= np.expand_dims(preprocess(camera.value),0)
    input_index = interpreter.get_input_details()[0]["index"]
    interpreter.set_tensor(input_index, input_tensor)
    interpreter.invoke()
    output_details = interpreter.get_output_details()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    prediction = np.squeeze(output_data)
    a=prediction.argmax(0)
    classfication_result_image.value = label[a]
    
mask = execute({'new': camera.value})
camera.observe(execute, names='value')

In [ ]:
camera.stop()